In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vaibhav1202","key":"d2371c72f7759a41acca8d403cb56776"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [0]:
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json

total 4
-rw------- 1 root root 67 Apr 23 12:09 kaggle.json
{"username":"vaibhav1202","key":"d2371c72f7759a41acca8d403cb56776"}

In [0]:
!pip install -q kaggle
!pip install -q kaggle-cli

    100% |████████████████████████████████| 81kB 3.8MB/s 
    100% |████████████████████████████████| 5.3MB 4.5MB/s 
    100% |████████████████████████████████| 102kB 28.1MB/s 
    100% |████████████████████████████████| 51kB 18.2MB/s 


In [0]:
!kaggle competitions download -c digit-recognizer

 93% 68.0M/73.2M [00:00<00:00, 120MB/s]
100% 73.2M/73.2M [00:00<00:00, 165MB/s]
 94% 46.0M/48.8M [00:00<00:00, 74.2MB/s]
100% 48.8M/48.8M [00:00<00:00, 96.5MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 69.3MB/s]


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [0]:
import pandas as pd
train= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")

In [0]:
train["label"].unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6])

In [0]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
Y_train=train["label"]
X_train=train.drop(["label"],axis="columns") 
Y_train = to_categorical(Y_train, num_classes = 10)

In [0]:
X_train = X_train / 255
test = test / 255

In [0]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [0]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [0]:
optimizer = "adam"
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
epochs = 20
batch_size = 64
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
final = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, Y_val),callbacks=[learning_rate_reduction])
results = model.predict(test)

Instructions for updating:
Use tf.cast instead.
Train on 37800 samples, validate on 4200 samples
Epoch 1/20
37800/37800 [==============================] - 274s 7ms/step - loss: 0.2423 - acc: 0.9226 - val_loss: 0.0843 - val_acc: 0.9755
Epoch 2/20
37800/37800 [==============================] - 274s 7ms/step - loss: 0.0831 - acc: 0.9744 - val_loss: 0.0432 - val_acc: 0.9864
Epoch 3/20
37800/37800 [==============================] - 272s 7ms/step - loss: 0.0599 - acc: 0.9824 - val_loss: 0.0334 - val_acc: 0.9912
Epoch 4/20
37800/37800 [==============================] - 272s 7ms/step - loss: 0.0506 - acc: 0.9838 - val_loss: 0.0419 - val_acc: 0.9900
Epoch 5/20
37800/37800 [==============================] - 273s 7ms/step - loss: 0.0425 - acc: 0.9870 - val_loss: 0.0352 - val_acc: 0.9907
Epoch 6/20
37800/37800 [==============================] - 273s 7ms/step - loss: 0.0378 - acc: 0.9883 - val_loss: 0.0291 - val_acc: 0.9912

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.000500000023748

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test, steps = 10)
print("loss",test_loss)
print("acc",test_acc)

NameError: ignored

In [0]:
test2 = tf.reshape(test, [-1, 28, 28,1])
prd = model.predict(test2, steps=10)
print(prd)

In [0]:
prd = np.argmax(prd, axis=1)
print(prd)

In [0]:
df3 =  pd.read_csv("sample_submission.csv")
df3.info()
df3["Label"]=pd.Series(prd)
df3.head()